In [14]:
!which python

//anaconda/envs/spark/bin/python


In [16]:
!python --version

Python 2.7.13 :: Continuum Analytics, Inc.


In [1]:
# make sure SparkContext is loaded 
sc

In [2]:
# load library 
import pandas as pd, numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [5]:
# load applications  
conf = SparkConf().setAppName("building a warehouse")
#sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
print (sc)

### 1) Read csv via spark SQL

In [7]:
# read csv via spark SQL  
df_train = sqlContext.read.format('com.databricks.spark.csv')\
						 .options(header='true', inferschema='true')\
						 .load('../data/train.csv')

In [13]:
df_train.show()

+---------+---------+--------------------+--------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|       id|vendor_id|     pickup_datetime|    dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+--------------------+--------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:...|2016-03-14 17:32:...|              1| -73.9821548461914| 40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|
|id2377394|        1|2016-06-12 00:43:...|2016-06-12 00:54:...|              1|-73.98041534423828|40.738563537597656|-73.99948120117188| 40.73115158081055|                 N|          663|
|id3858529|        2|2016-01-19 11:35:...|2016-01-19 12

In [12]:
df_train.take(4)

[Row(id=u'id2875421', vendor_id=2, pickup_datetime=datetime.datetime(2016, 3, 14, 17, 24, 55), dropoff_datetime=datetime.datetime(2016, 3, 14, 17, 32, 30), passenger_count=1, pickup_longitude=-73.9821548461914, pickup_latitude=40.76793670654297, dropoff_longitude=-73.96463012695312, dropoff_latitude=40.765602111816406, store_and_fwd_flag=u'N', trip_duration=455),
 Row(id=u'id2377394', vendor_id=1, pickup_datetime=datetime.datetime(2016, 6, 12, 0, 43, 35), dropoff_datetime=datetime.datetime(2016, 6, 12, 0, 54, 38), passenger_count=1, pickup_longitude=-73.98041534423828, pickup_latitude=40.738563537597656, dropoff_longitude=-73.99948120117188, dropoff_latitude=40.73115158081055, store_and_fwd_flag=u'N', trip_duration=663),
 Row(id=u'id3858529', vendor_id=2, pickup_datetime=datetime.datetime(2016, 1, 19, 11, 35, 24), dropoff_datetime=datetime.datetime(2016, 1, 19, 12, 10, 48), passenger_count=1, pickup_longitude=-73.9790267944336, pickup_latitude=40.763938903808594, dropoff_longitude=-74.

In [32]:
df_train.registerTempTable("df_train_table")
sqlContext.sql("""
                SELECT id, count(*) 
                FROM df_train_table
                group by 1 
                order by 2 desc 
                limit 10""").show()


+---------+--------+
|       id|count(1)|
+---------+--------+
|id0512853|       1|
|id1924450|       1|
|id0707492|       1|
|id0544194|       1|
|id3129935|       1|
|id2816240|       1|
|id1921215|       1|
|id0306216|       1|
|id0486362|       1|
|id1442965|       1|
+---------+--------+



### 2) basic functions 

In [59]:
df__ = sc.textFile("../data/train.csv")
#df__.filter(lambda x: '2124' in x.).collect()

In [36]:
df__.take(5)

[u'id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration',
 u'id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191406,40.767936706542969,-73.964630126953125,40.765602111816406,N,455',
 u'id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238281,40.738563537597656,-73.999481201171875,40.731151580810547,N,663',
 u'id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433594,40.763938903808594,-74.005332946777344,40.710086822509766,N,2124',
 u'id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125,40.719970703125,-74.01226806640625,40.706718444824219,N,429']

In [58]:
#protocols = df__.map(lambda x: x[4]).distinct()
#protocols.take(10)


In [56]:
df__.map(lambda x: x[0:20]).distinct().take(10)
    
#.map(lambda x: x[1]).distinct()

[u'id2922821,1,2016-05-',
 u'id1145978,1,2016-06-',
 u'id1722464,1,2016-03-',
 u'id2258021,1,2016-03-',
 u'id1476617,2,2016-06-',
 u'id3972810,1,2016-05-',
 u'id3173897,1,2016-03-',
 u'id1492272,2,2016-01-',
 u'id1831578,1,2016-04-',
 u'id1646416,1,2016-03-']

In [57]:
# df__.map(lambda x: x[12:22]).take(10)

In [43]:
df__.map(lambda x: x[:]).take(10)

[u'id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration',
 u'id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191406,40.767936706542969,-73.964630126953125,40.765602111816406,N,455',
 u'id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238281,40.738563537597656,-73.999481201171875,40.731151580810547,N,663',
 u'id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433594,40.763938903808594,-74.005332946777344,40.710086822509766,N,2124',
 u'id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125,40.719970703125,-74.01226806640625,40.706718444824219,N,429',
 u'id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973052978515625,40.793209075927734,-73.972923278808594,40.782520294189453,N,435',
 u'id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982856750488281,40.742195129394531,-73.992080688476562,40.7